# Test mutual information estimators

## Preamble

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

tfds.disable_progress_bar()
tf.enable_v2_behavior()

import logging
tf.get_logger().setLevel(logging.ERROR)

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
font = {'family' : 'DejaVu Sans',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
import os
import json
import csv

from datetime import datetime

In [ ]:
from pathlib import Path
path = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), "../../data/"))

In [ ]:
experiments_path = path + "/mutual_information/synthetic/"

#### Importing the module

In [ ]:
import mutinfo.estimators.mutual_information as mi_estimators
from mutinfo.utils.dependent_norm import multivariate_normal_from_MI

In [ ]:
### SETTINGS ###
%run ./Settings.ipynb

#### Standard tests with arbitrary mapping

In [ ]:
def perform_normal_compressed_test(mi, n_samples, X_dimension, Y_dimension, X_map=None, Y_map=None,
                                   X_compressor=None, Y_compressor=None, verbose=0):
    # Generation.
    random_variable = multivariate_normal_from_MI(X_dimension, Y_dimension, mi)
    X_Y = random_variable.rvs(n_samples)
    X = X_Y[:, 0:X_dimension]
    Y = X_Y[:, X_dimension:X_dimension + Y_dimension]
        
    # Mapping application.
    if not X_map is None:
        X = X_map(X)
           
    if not Y_map is None:
        Y = Y_map(Y)
        
    # Mutual information estimation.
    mi_estimator = mi_estimators.MutualInfoEstimator(entropy_estimator_params=entropy_estimator_params)
    mi_estimator.fit(X, Y, verbose=verbose)
    mi = mi_estimator.estimate(X, Y, verbose=verbose)
    
    # Mutual information estimation for compressed representation.
    mi_estimator = mi_estimators.LossyMutualInfoEstimator(X_compressor, Y_compressor,
                                                          entropy_estimator_params=entropy_estimator_params)
    mi_estimator.fit(X, Y, verbose=verbose)
    mi_compressed = mi_estimator.estimate(X, Y, verbose=verbose)
    
    return mi, mi_compressed

In [ ]:
def perform_normal_compressed_tests_MI(MI, n_samples, X_dimension, Y_dimension, X_map=None, Y_map=None,
                                       X_compressor=None, Y_compressor=None, verbose=0):
    """
    Estimate mutual information for different true values
    (transformed normal distribution).
    """
    n_exps = len(MI)
    
    # Mutual information estimates.
    estimated_MI = []
    estimated_MI_compressed = []

    # Conducting the tests.
    for n_exp in range(n_exps):
        print("\nn_exp = %d/%d\n------------\n" % (n_exp + 1, n_exps))
        mi, compressed_mi = perform_normal_compressed_test(MI[n_exp], n_samples, X_dimension, Y_dimension,
                                                           X_map, Y_map, X_compressor, Y_compressor, verbose)
        estimated_MI.append(mi)
        estimated_MI_compressed.append(compressed_mi)
        
    return estimated_MI, estimated_MI_compressed

In [ ]:
def plot_estimated_compressed_MI(MI, estimated_MI, estimated_MI_compressed, title):
    estimated_MI_mean = np.array([estimated_MI[index][0] for index in range(len(estimated_MI))])
    estimated_MI_std  = np.array([estimated_MI[index][1] for index in range(len(estimated_MI))])
    
    estimated_MI_compressed_mean = np.array([estimated_MI_compressed[index][0]
                                             for index in range(len(estimated_MI_compressed))])
    estimated_MI_compressed_std  = np.array([estimated_MI_compressed[index][1]
                                             for index in range(len(estimated_MI_compressed))])
    
    fig_normal, ax_normal = plt.subplots()

    fig_normal.set_figheight(11)
    fig_normal.set_figwidth(16)

    # Grid.
    ax_normal.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
    ax_normal.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

    ax_normal.set_title(title)
    ax_normal.set_xlabel("$I(X,Y)$")
    ax_normal.set_ylabel("$\\hat I(X,Y)$")
    
    ax_normal.minorticks_on()
    
    #ax_normal.set_yscale('log')
    #ax_normal.set_xscale('log')

    ax_normal.plot(MI, MI, label="$I(X,Y)$", color='red')
    
    ax_normal.plot(MI, estimated_MI_mean, label="$\\hat I(X,Y)$")
    ax_normal.fill_between(MI, estimated_MI_mean + estimated_MI_std, estimated_MI_mean - estimated_MI_std, alpha=0.2)
    
    ax_normal.plot(MI, estimated_MI_compressed_mean, label="$\\hat I_{compr}(X,Y)$")
    ax_normal.fill_between(MI, estimated_MI_compressed_mean + estimated_MI_compressed_std,
                           estimated_MI_compressed_mean - estimated_MI_compressed_std, alpha=0.2)

    ax_normal.legend(loc='upper left')

    ax_normal.set_xlim((0.0, None))
    ax_normal.set_ylim((0.0, None))

    plt.show();

### Global parameters

In [ ]:
# The values of mutual information under study.
MI = np.linspace(0.0, 10.0, 41)
n_exps = len(MI)

# Sample size and dimensions of vectors X and Y.
n_samples = 5000

### Images of correlated gaussians

In [ ]:
from mutinfo.utils.synthetic import *

In [ ]:
X_dimension = 5
Y_dimension = 5
latent_dimension = 5

img_width = 32
img_height = 32

experiments_dir = ('gaussian_correlated_%dx%d' % (img_width, img_height))

In [ ]:
def imshow_array(array):
    """Display array of pixels."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

#### Train the autoencoder

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
n_train_samples = 6000
n_test_samples  = 1000

In [ ]:
random_variable = multivariate_normal()
X = random_variable.rvs((n_train_samples + n_test_samples, X_dimension))
X = normal_to_uniform(X)

distribution = lambda X, Y, params : np.exp(
    -10.0 * (
        (1.0 + params[:,2,None,None])**2 * (X - params[:,0,None,None])**2 +
        (1.0 + params[:,3,None,None])**2 * (Y - params[:,1,None,None])**2 +
        (-1.0 + 2.0 * params[:,4,None,None])*(1.0 + params[:,2,None,None])*(1.0 + params[:,3,None,None]) * 
        (X - params[:,0,None,None])*(Y - params[:,1,None,None])
    )
)

X = params_to_2d_distribution(X, distribution, img_width, img_height)
X = np.expand_dims(X, axis=-1)
X_train = X[0:n_train_samples]
X_test  = X[n_train_samples:n_train_samples + n_test_samples]

In [ ]:
X_dataset = tf.data.Dataset.from_tensor_slices(X_train)

In [ ]:
augmentator = tf.keras.Sequential([
    tf.keras.layers.Input((img_width, img_height, 1)),
    #tf.keras.layers.RandomTranslation(
    #    height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode="constant"
    #),
    tf.keras.layers.RandomZoom(
        height_factor=(-0.2, 0.0), width_factor=(-0.2, 0.0), fill_mode="constant"
    )
])
augmentator.compile()

def augment(sample):
    sample = augmentator(sample, training=True)
    return sample, sample

In [ ]:
imshow_array(augment(X[0][None,])[0].numpy()[0,:,:,0])

In [ ]:
X_augmented_dataset = X_dataset.shuffle(10000).batch(5000).map(augment, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def cnn_autoencoder(shape_input, dimension):
    # Weight initialization.
    init = tf.keras.initializers.RandomNormal(stddev=1e-1)

    # Input data.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer
    
    # 1 block of layers.
    next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.2)(next_layer)

    # 2 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.1)(next_layer)
    
    # 3 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.2)(next_layer)
    
    # 4 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.2)(next_layer)
    
    # 5 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.2)(next_layer)
    
    # 6 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')(next_layer)
    #next_layer = tf.keras.layers.Dropout(rate=0.2)(next_layer)

    # Bottleneck.
    next_layer = tf.keras.layers.Flatten()(next_layer)
    next_layer = tf.keras.layers.Dense(dimension, kernel_initializer=init)(next_layer)
    #next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    bottleneck = tf.keras.layers.Activation('tanh')(next_layer)

    # Encoder model.
    encoder = tf.keras.Model(input_layer, bottleneck)

    # Decoder model begins.
    input_code_layer = tf.keras.layers.Input((dimension))
    next_layer = input_code_layer
    next_layer = tf.keras.layers.GaussianNoise(0.02)(next_layer)
    
    # 6 block of layers.
    #tfa.layers.SpectralNormalization()
    next_layer = tf.keras.layers.Dense(1*1*8, kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.Reshape((1, 1, 8))(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 5 block of layers.
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 4 block of layers.
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 3 block of layers.
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 2 block of layers.
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 1 block of layers.
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 0 block of layers.
    next_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer=init)(next_layer)
    #next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.Activation('sigmoid')(next_layer)

    output_layer = next_layer

    # Model.
    decoder = tf.keras.models.Model(input_code_layer, output_layer) # Decoder.
    autoencoder = tf.keras.Sequential([encoder, decoder])

    # Compile the model.
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
    autoencoder.compile(loss='mae', optimizer=opt)
    return encoder, decoder, autoencoder

In [ ]:
load_autoencoder = True
models_path_ = experiments_path + experiments_dir + "/models/autoencoder/"

In [ ]:
if load_autoencoder:
    encoder = tf.keras.models.load_model(models_path_ + "encoder.h5")
    decoder = tf.keras.models.load_model(models_path_ + "decoder.h5")
    autoencoder = tf.keras.Sequential([encoder, decoder])
    autoencoder.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

In [ ]:
if not load_autoencoder:
    encoder, decoder, autoencoder = cnn_autoencoder((img_width, img_height, 1), latent_dimension)
    
    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            fig, ax = plt.subplots(2, 2)
            fig.set_figheight(8)
            fig.set_figwidth(8)
            
            ax[0][0].axis('off')
            ax[0][1].axis('off')
            ax[1][0].axis('off')
            ax[1][1].axis('off')
            
            ax[0][0].imshow(X_test[0], cmap=plt.get_cmap("gray"), vmin=0.0, vmax=1.0)
            ax[0][1].imshow(autoencoder(X_test[0:1]).numpy()[0,:,:,0],
                         cmap=plt.get_cmap("gray"), vmin=0.0, vmax=1.0)
            
            sample = next(iter(X_augmented_dataset))[0]
            
            ax[1][0].imshow(sample.numpy()[0,:], cmap=plt.get_cmap("gray"), vmin=0.0, vmax=1.0)
            ax[1][1].imshow(autoencoder(sample).numpy()[0,:,:,0],
                         cmap=plt.get_cmap("gray"), vmin=0.0, vmax=1.0)
            plt.show();
    
    autoencoder.fit(
        X_augmented_dataset,
        epochs=2,
        validation_data=(X_test, X_test),
        callbacks=[CustomCallback()],
    )
    
    # Save the models.
    os.makedirs(models_path_, exist_ok=True)
    autoencoder.save(models_path_ + "autoencoder.h5")
    encoder.save(models_path_ + "encoder.h5")
    decoder.save(models_path_ + "decoder.h5")

In [ ]:
autoencoder.summary()

In [ ]:
def gaussian_mapping(X):
    """ Map Gaussian vector to the coordiantes of the mode (center of the plot) and covariance matrix. """
    return normal_to_uniform(X)

def gaussian_compressor(X):
    """ Parameters to images, then to latent representations. """
    return encoder(np.expand_dims(params_to_2d_distribution(X, distribution, img_width, img_height),
                                  axis=-1)).numpy()

In [ ]:
estimated_MI, estimated_MI_compressed = perform_normal_compressed_tests_MI(MI,
    n_samples, X_dimension, Y_dimension, gaussian_mapping, gaussian_mapping,
    gaussian_compressor, gaussian_compressor, verbose=10)

In [ ]:
plot_estimated_compressed_MI(MI, estimated_MI, estimated_MI_compressed, "Correlated 2D Gaussians")

In [ ]:
save_estimated_MI(MI, estimated_MI, experiments_dir + '/parameters')
save_estimated_MI(MI, estimated_MI_compressed, experiments_dir + '/compressed')

In [ ]:
print("OK")